In [36]:
import requests
import psycopg2
import pandas as pd


In [37]:
NEON_DB_CONFIG = {
    "host": "ep-raspy-night-ah8kwb6a-pooler.c-3.us-east-1.aws.neon.tech",
    "database": "neondb",
    "user": "neondb_owner",
    "password": "npg_tGk50unxySTW",
    "port": "5432",
    "sslmode": "require"
}







In [38]:
def get_neon_connection():
    return psycopg2.connect(
        host=NEON_DB_CONFIG["host"],
        database=NEON_DB_CONFIG["database"],
        user=NEON_DB_CONFIG["user"],
        password=NEON_DB_CONFIG["password"],
        port=NEON_DB_CONFIG["port"],
        sslmode=NEON_DB_CONFIG["sslmode"]
    )


In [39]:
SCHEMA_PROMPT = """
You are an expert PostgreSQL SQL generator.

Database schema:

table_first(
    f_date DATE,
    firm VARCHAR,
    firm_product VARCHAR,
    voucher_type VARCHAR,
    voucher_no VARCHAR,
    gstin_uin VARCHAR,
    quantity INT,
    rate NUMERIC,
    value NUMERIC,
    brand VARCHAR,
    sub_category VARCHAR,
    category VARCHAR,
    client_type VARCHAR
)

table_second(
    date_text TIMESTAMP,
    firm VARCHAR,
    voucher_no VARCHAR,
    quantity INT,
    value NUMERIC,
    gstin_uin VARCHAR,
    total_tax_amount NUMERIC,
    gross_total NUMERIC
)

Relationships:
- table_first.firm = table_second.firm
- table_first.voucher_no = table_second.voucher_no

Rules:
- Generate valid PostgreSQL SQL
- Use JOIN only if needed
- Return ONLY SQL
"""


In [68]:
def nl_to_sql_llama(question):
    url = "http://localhost:11434/api/generate"

    payload = {
        "model": "llama3:8b",
        "prompt": SCHEMA_PROMPT + "\nUser question:\n" + question,
        "stream": False,
        "options": {"temperature": 0}
    }

    response = requests.post(url, json=payload, timeout=300)
    data = response.json()

    raw_response = data["response"]
    print("\n🧠 Raw LLaMA Output:\n", raw_response)

    clean_sql = extract_sql(raw_response)
    print("\n🧼 Clean SQL:\n", clean_sql)

    return clean_sql


In [70]:
def execute_sql(sql_query):
    conn = get_neon_connection()
    try:
        df = pd.read_sql(sql_query, conn)
        conn.close()
        return df
    except Exception as e:
        conn.close()
        return str(e)


In [52]:
def ask_database(question):
    print("🟢 Question:")
    print(question)

    sql_query = nl_to_sql_llama(question)
    print("\n🟡 Generated SQL:")
    print(sql_query)

    result = execute_sql(sql_query)

    if isinstance(result, str):
        print("\n🔴 Error:")
        print(result)
    else:
        display(result)


In [72]:
ask_database("Show total sales value by brand")


🟢 Question:
Show total sales value by brand

🧠 Raw LLaMA Output:
 Here is the PostgreSQL SQL to show total sales value by brand:

```sql
SELECT 
    brand, 
    SUM(value) AS total_sales_value
FROM 
    table_first
GROUP BY 
    brand;
```

This query uses a simple `GROUP BY` clause to group the results by the `brand` column and then calculates the sum of the `value` column for each group using the `SUM` aggregation function.

🧼 Clean SQL:
 SELECT 
    brand, 
    SUM(value) AS total_sales_value
FROM 
    table_first
GROUP BY 
    brand;

🟡 Generated SQL:
SELECT 
    brand, 
    SUM(value) AS total_sales_value
FROM 
    table_first
GROUP BY 
    brand;


C:\Users\kudal\AppData\Local\Temp\ipykernel_10528\1369160556.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql_query, conn)


,brand,total_sales_value
0,Other,15836155.12
1,Prama,1255107.10
2,Beetel,16495.85
3,Irange,11027.22
4,Daichi,631771.01
5,Hikvision,5972730.88
6,CP Plus,859369.74


In [66]:
import re

def extract_sql(text):
    """
    Extracts SQL code from LLM response.
    Handles ```sql blocks and plain SQL.
    """
    # Case 1: SQL inside ```sql ... ```
    match = re.search(r"```sql(.*?)```", text, re.DOTALL | re.IGNORECASE)
    if match:
        return match.group(1).strip()

    # Case 2: Starts directly with SELECT
    if text.strip().lower().startswith("select"):
        return text.strip()

    # Case 3: Fallback (try to find SELECT ... ;)
    match = re.search(r"(select .*?;)", text, re.DOTALL | re.IGNORECASE)
    if match:
        return match.group(1).strip()

    return text.strip()


In [74]:
ask_database("highest tax ammount")

🟢 Question:
highest tax ammount

🧠 Raw LLaMA Output:
 Here is the PostgreSQL SQL to answer the user's question:

```sql
SELECT SUM(total_tax_amount) AS highest_tax_amount
FROM table_second;
```

This query simply sums up all the `total_tax_amount` values in `table_second`, which gives us the total tax amount.

🧼 Clean SQL:
 SELECT SUM(total_tax_amount) AS highest_tax_amount
FROM table_second;

🟡 Generated SQL:
SELECT SUM(total_tax_amount) AS highest_tax_amount
FROM table_second;


C:\Users\kudal\AppData\Local\Temp\ipykernel_10528\1369160556.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql_query, conn)


,highest_tax_amount
0,986824.02


In [76]:
ask_database("lowest tax amount")

🟢 Question:
lowest tax amount

🧠 Raw LLaMA Output:
 Here is the PostgreSQL SQL to answer the user's question:

```sql
SELECT MIN(total_tax_amount) 
FROM table_second;
```

This query will return the lowest tax amount from `table_second`. If you want to filter the result by a specific firm or voucher_no, you can add a WHERE clause as follows:

```sql
SELECT MIN(total_tax_amount) 
FROM table_second 
WHERE firm = 'your_firm' AND voucher_no = 'your_voucher_no';
```

Replace `'your_firm'` and `'your_voucher_no'` with the actual values you want to filter by.

🧼 Clean SQL:
 SELECT MIN(total_tax_amount) 
FROM table_second;

🟡 Generated SQL:
SELECT MIN(total_tax_amount) 
FROM table_second;


C:\Users\kudal\AppData\Local\Temp\ipykernel_10528\1369160556.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql_query, conn)


,min
0,0.0


In [78]:
ask_database("seprate common firm from both")

🟢 Question:
seprate common firm from both

🧠 Raw LLaMA Output:
 Here is the PostgreSQL SQL to separate common firms from both tables:

```sql
SELECT DISTINCT tf.firm 
FROM table_first tf 
WHERE tf.firm IN (SELECT ts.firm FROM table_second ts);
```

This query uses a subquery to get all unique firms from `table_second` and then selects only those firms that exist in `table_first`. The `DISTINCT` keyword is used to ensure that each firm is only returned once.

🧼 Clean SQL:
 SELECT DISTINCT tf.firm 
FROM table_first tf 
WHERE tf.firm IN (SELECT ts.firm FROM table_second ts);

🟡 Generated SQL:
SELECT DISTINCT tf.firm 
FROM table_first tf 
WHERE tf.firm IN (SELECT ts.firm FROM table_second ts);


C:\Users\kudal\AppData\Local\Temp\ipykernel_10528\1369160556.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql_query, conn)


,firm
0,SHRI SHYAM COMPUTERS
1,Tutus India Automation Private Limited
2,FALCON SECURITY SOLUTIONS
3,RAJ KUMAR
4,PTECHIES SECURITY SYSTEMS
...,...
190,SAHIL GURGAON
191,SMART CCTV CAMERA INSTALLER
192,JSR INFOTECH
193,G M SECURITECH


In [80]:
ask_database("from table first findhighest quantity")

🟢 Question:
from table first findhighest quantity

🧠 Raw LLaMA Output:
 Here is the PostgreSQL SQL to find the highest quantity from `table_first`:

```sql
SELECT MAX(quantity) AS highest_quantity
FROM table_first;
```

This query uses the `MAX` aggregation function to find the maximum value in the `quantity` column. The result is returned as a single row with a single column named `highest_quantity`.

🧼 Clean SQL:
 SELECT MAX(quantity) AS highest_quantity
FROM table_first;

🟡 Generated SQL:
SELECT MAX(quantity) AS highest_quantity
FROM table_first;


C:\Users\kudal\AppData\Local\Temp\ipykernel_10528\1369160556.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql_query, conn)


,highest_quantity
0,11778


In [84]:
ask_database("sub-category wise revenue")


🟢 Question:
sub-category wise revenue

🧠 Raw LLaMA Output:
 Here is the PostgreSQL SQL to generate sub-category wise revenue:

```sql
SELECT 
    t1.sub_category, 
    SUM(t1.value) AS revenue
FROM 
    table_first t1
GROUP BY 
    t1.sub_category;
```

This query uses a simple `GROUP BY` clause to group the data by `sub_category`, and then calculates the sum of `value` for each sub-category using the `SUM` aggregation function.

🧼 Clean SQL:
 SELECT 
    t1.sub_category, 
    SUM(t1.value) AS revenue
FROM 
    table_first t1
GROUP BY 
    t1.sub_category;

🟡 Generated SQL:
SELECT 
    t1.sub_category, 
    SUM(t1.value) AS revenue
FROM 
    table_first t1
GROUP BY 
    t1.sub_category;


C:\Users\kudal\AppData\Local\Temp\ipykernel_10528\1369160556.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql_query, conn)


,sub_category,revenue
0,Access Control Unit,22610.19
1,HDMI Cable,11449.13
2,3+1 Cable,79474.72
3,Network Switch,59923.63
4,Camera,5818580.82
5,SMPS,224326.13
6,Door Lock,76796.46
7,Rack,20101.58
8,Other Accessories,14121894.42
9,Connector,17131.47


In [86]:
ask_database("sub-category wise total tax ammount")

🟢 Question:
sub-category wise total tax ammount

🧠 Raw LLaMA Output:
 Here is the PostgreSQL SQL to answer the user's question:

```sql
SELECT 
    sub_category, 
    SUM(total_tax_amount) AS total_tax_amount
FROM 
    table_second
JOIN 
    table_first ON table_second.firm = table_first.firm AND table_second.voucher_no = table_first.voucher_no
GROUP BY 
    sub_category;
```

This SQL uses a JOIN to combine the two tables based on the firm and voucher_no columns. It then groups the results by the sub_category column and calculates the total tax amount for each sub-category using the SUM function.

🧼 Clean SQL:
 SELECT 
    sub_category, 
    SUM(total_tax_amount) AS total_tax_amount
FROM 
    table_second
JOIN 
    table_first ON table_second.firm = table_first.firm AND table_second.voucher_no = table_first.voucher_no
GROUP BY 
    sub_category;

🟡 Generated SQL:
SELECT 
    sub_category, 
    SUM(total_tax_amount) AS total_tax_amount
FROM 
    table_second
JOIN 
    table_first ON ta

C:\Users\kudal\AppData\Local\Temp\ipykernel_10528\1369160556.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(sql_query, conn)


,sub_category,total_tax_amount
0,3+1 Cable,126960.74
1,Access Control Unit,106083.20
2,CAT 6 Cable,116189.84
3,Cable,97535.32
4,Camera,1762611.36
5,Connector,107716.58
6,Door Lock,179277.46
7,HDMI Cable,307444.22
8,Hard Drive,407851.40
9,Monitor,19453.54
